In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [4]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/up-btc/upbit-2018-12-btc-krw.csv")

In [5]:
df.dtypes

timestamp     object
quantity     float64
price          int64
fee          float64
amount         int64
side           int64
dtype: object

In [6]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price                 int64
fee                 float64
amount                int64
side                  int64
dtype: object

In [7]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [8]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2018-12-02 01:27:00,0.002100,4817000,5.05,-10121,0,2018-12-02,2018,12,December,2,01:27:00,1,27,0
1,2018-12-02 01:30:00,0.002100,4821000,5.06,10119,1,2018-12-02,2018,12,December,2,01:30:00,1,30,0
2,2018-12-02 02:02:00,0.448918,4816000,1080.99,-2163071,0,2018-12-02,2018,12,December,2,02:02:00,2,2,0
3,2018-12-02 02:02:00,0.009058,4816000,21.81,-43646,0,2018-12-02,2018,12,December,2,02:02:00,2,2,0
4,2018-12-02 02:02:00,0.115024,4816000,276.97,-554232,0,2018-12-02,2018,12,December,2,02:02:00,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,2018-12-28 09:34:00,0.058000,4107000,119.10,238086,1,2018-12-28,2018,12,December,28,09:34:00,9,34,0
3668,2018-12-28 09:35:00,0.004300,4101000,8.81,17625,1,2018-12-28,2018,12,December,28,09:35:00,9,35,0
3669,2018-12-28 09:35:00,0.980000,4101000,2009.49,4016970,1,2018-12-28,2018,12,December,28,09:35:00,9,35,0
3670,2018-12-28 09:41:00,0.041716,4093000,85.37,170660,1,2018-12-28,2018,12,December,28,09:41:00,9,41,0


In [9]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [10]:
group_day_amount.size()

timestamp_year_month_day
2018-12-02     334
2018-12-03     246
2018-12-19     125
2018-12-20    1045
2018-12-21    1406
2018-12-22     201
2018-12-27     140
2018-12-28     175
Name: amount, dtype: int64

In [11]:
group_day_amount

In [12]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2018-12-02    -170441
2018-12-03     175400
2018-12-19      48014
2018-12-20   -6385830
2018-12-21    6321246
2018-12-22    -179293
2018-12-27     -46316
2018-12-28    -446386
Name: amount, dtype: int64

In [13]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [14]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2018-12-02    211
2018-12-03    162
2018-12-19     61
2018-12-20    600
2018-12-21    721
2018-12-22    135
2018-12-27     57
2018-12-28    134
Name: amount, dtype: int64

In [15]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2018-12-02    190655671
2018-12-03    181207581
2018-12-19     59819114
2018-12-20    511010007
2018-12-21    623223575
2018-12-22    108705314
2018-12-27     55045007
2018-12-28    119682998
Name: amount, dtype: int64

In [16]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [17]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2018-12-02    123
2018-12-03     83
2018-12-19     64
2018-12-20    444
2018-12-21    685
2018-12-22     66
2018-12-27     83
2018-12-28     41
Name: amount, dtype: int64

In [18]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2018-12-02   -190826112
2018-12-03   -181032181
2018-12-19    -59771100
2018-12-20   -517395837
2018-12-21   -616902329
2018-12-22   -108884607
2018-12-27    -55091323
2018-12-28   -120129384
Name: amount, dtype: int64

In [19]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2018-12-02     381481783
2018-12-03     362239762
2018-12-19     119590214
2018-12-20    1028405844
2018-12-21    1240125904
2018-12-22     217589921
2018-12-27     110136330
2018-12-28     239812382
Name: amount, dtype: int64

In [20]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2018-12-02     334
 2018-12-03     246
 2018-12-19     125
 2018-12-20    1045
 2018-12-21    1406
 2018-12-22     201
 2018-12-27     140
 2018-12-28     175
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-12-02    -170441
 2018-12-03     175400
 2018-12-19      48014
 2018-12-20   -6385830
 2018-12-21    6321246
 2018-12-22    -179293
 2018-12-27     -46316
 2018-12-28    -446386
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-12-02    211
 2018-12-03    162
 2018-12-19     61
 2018-12-20    600
 2018-12-21    721
 2018-12-22    135
 2018-12-27     57
 2018-12-28    134
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-12-02    190655671
 2018-12-03    181207581
 2018-12-19     59819114
 2018-12-20    511010007
 2018-12-21    623223575
 2018-12-22    108705314
 2018-12-27     55045007
 2018-12-28    119682998
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-12-02    123
 2018-12-03     83
 2018-12-19     64
 

In [21]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-12-02,334,-170441,211,190655671,123,-190826112,381481783,190826112
2018-12-03,246,175400,162,181207581,83,-181032181,362239762,181032181
2018-12-19,125,48014,61,59819114,64,-59771100,119590214,59771100
2018-12-20,1045,-6385830,600,511010007,444,-517395837,1028405844,517395837
2018-12-21,1406,6321246,721,623223575,685,-616902329,1240125904,616902329
2018-12-22,201,-179293,135,108705314,66,-108884607,217589921,108884607
2018-12-27,140,-46316,57,55045007,83,-55091323,110136330,55091323
2018-12-28,175,-446386,134,119682998,41,-120129384,239812382,120129384


In [22]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2018-12-02                            334.0     -170441.0   
2018-12-03                            246.0      175400.0   
2018-12-04                              NaN           NaN   
2018-12-05                              NaN           NaN   
2018-12-06                              NaN           NaN   
2018-12-07                              NaN           NaN   
2018-12-08                              NaN           NaN   
2018-12-09                              NaN           NaN   
2018-12-10                              NaN           NaN   
2018-12-11                              NaN           NaN   
2018-12-12                              NaN           NaN   
2018-12-13                              NaN           NaN   
2018-12-14                              NaN           NaN   
2018-12-15                              NaN           NaN   
2018-12-16              

In [23]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/upbit-2018-12-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [24]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/upbit-2018-12-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-01-19,40,1052348,21,1334332,19,-281984,1616316.0,281984
2018-01-20,153,426,76,1239191,77,-1238765,2477956.0,1238765
2018-01-21,154,-1741,76,1201705,78,-1203446,2405151.0,1203446
2018-01-22,69,-14922,34,480676,35,-495598,976274.0,495598
2018-01-23,96,899,48,633072,48,-632173,1265245.0,632173
2018-01-24,86,-1680,42,565300,44,-566980,1132280.0,566980
2018-01-25,217,11247,109,1442316,108,-1431069,2873385.0,1431069
2018-01-31,4,-86,2,22176,2,-22262,44438.0,22262
